#간단한 Convolutional Encoder-Decoder 네트워크

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from datetime import datetime


In [15]:
class SequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.encoder = keras.Sequential(
        [
         layers.Conv2D(16, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='en1'),
         layers.Conv2D(32, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first',  name='en2'),
         layers.Conv2D(64, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first',  name='en3'),
         layers.Conv2D(128, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first',  name='en4'),
         layers.Conv2D(128, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first',  name='en5'),
         layers.Conv2D(128, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first',  name='en6')
        ])

    self.decoder = keras.Sequential(
        [
         layers.Conv2DTranspose(128, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='de1'),
         layers.Conv2DTranspose(128, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='de2'),
         layers.Conv2DTranspose(64, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='de3'),
         layers.Conv2DTranspose(32, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='de4'),
         layers.Conv2DTranspose(16, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='de5'),
         layers.Conv2DTranspose(1, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='de6')
        ])
    

  def __call__(self, x):
    x = self.encoder(x)
    x = self.decoder(x)
    return x


In [17]:
# You have made a model!
my_model = SequentialModule(name="the_model")

# random input
input_shape = (1, 1, 256, 483)
input = tf.random.normal(input_shape)

# model output                    
output = my_model(input)

# summary
my_model.encoder.summary()
my_model.decoder.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 en1 (Conv2D)                (1, 16, 128, 483)         176       
                                                                 
 en2 (Conv2D)                (1, 32, 64, 483)          5152      
                                                                 
 en3 (Conv2D)                (1, 64, 32, 483)          20544     
                                                                 
 en4 (Conv2D)                (1, 128, 16, 483)         82048     
                                                                 
 en5 (Conv2D)                (1, 128, 8, 483)          163968    
                                                                 
 en6 (Conv2D)                (1, 128, 4, 483)          163968    
                                                                 
Total params: 435,856
Trainable params: 435,856
Non-t

# 간단한 Convolutional Encoder-Decoder 네트워크 2

In [18]:
from tensorflow import keras
from tensorflow.keras import layers

In [56]:
def encoder(x):
    x = layers.Conv2D(16, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='en1')(x)
    x = layers.Conv2D(32, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first',  name='en2')(x)
    x =  layers.Conv2D(64, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first',  name='en3')(x)
    x = layers.Conv2D(128, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first',  name='en4')(x)
    x = layers.Conv2D(128, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first',  name='en5')(x)
    x = layers.Conv2D(128, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first',  name='en6')(x)
    
    return x

In [57]:
def decoder(x):
    x = layers.Conv2DTranspose(128, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='de1')(x)
    x = layers.Conv2DTranspose(128, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='de2')(x)
    x = layers.Conv2DTranspose(64, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='de3')(x)
    x = layers.Conv2DTranspose(32, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='de4')(x)
    x = layers.Conv2DTranspose(16, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='de5')(x)
    x = layers.Conv2DTranspose(1, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='de6')(x)
    return x


In [64]:
def build_model(weight=None,
                input_shape=(1, 256, 483),
                model_name='encoder-decoder'):
  
    x = random_input = keras.Input(shape=input_shape)
    
    x = encoder(x)
    x = decoder(x)

    model = keras.Model(inputs=random_input, outputs=x, name=model_name)
    if weight is not None:
        model.load_weights(weight, by_name=True, skip_mismatch=True)
    return model

In [65]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    lr_schedule = keras.optimizers.schedules.CosineDecay(0.001, 0.9)
    model = build_model()
    model.compile(optimizer=keras.optimizers.Adam(lr_schedule),
                  loss=tf.keras.losses.MeanSquaredError())

model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Model: "encoder-decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 1, 256, 483)]     0         
                                                                 
 en1 (Conv2D)                (None, 16, 128, 483)

# 간단한 Convolutional Recurrent 네트워크


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def encoder(x):
    x = layers.Conv2D(16, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='en1')(x)
    x = layers.Conv2D(32, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first',  name='en2')(x)
    x =  layers.Conv2D(64, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first',  name='en3')(x)
    x = layers.Conv2D(128, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first',  name='en4')(x)
    x = layers.Conv2D(128, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first',  name='en5')(x)
    x = layers.Conv2D(128, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first',  name='en6')(x)
    
    return x

In [ ]:
def decoder(x):
    x = layers.Conv2DTranspose(128, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='de1')(x)
    x = layers.Conv2DTranspose(128, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='de2')(x)
    x = layers.Conv2DTranspose(64, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='de3')(x)
    x = layers.Conv2DTranspose(32, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='de4')(x)
    x = layers.Conv2DTranspose(16, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='de5')(x)
    x = layers.Conv2DTranspose(1, kernel_size=(5, 2), strides=(2, 1), padding='same', activation="relu", data_format='channels_first', name='de6')(x)
    return x


In [102]:
def lstm(x):
  x = layers.Reshape((483, -1))(x)
  x = layers.LSTM(256, activation='relu', return_sequences=True,)(x)
  x = layers.Dense(512)(x)
  x = layers.Reshape((128, 483, 4))(x)
  return x

In [68]:
def build_model(weight=None,
                input_shape=(1, 256, 483),
                model_name='encoder-decoder'):
  
    x = random_input = keras.Input(shape=input_shape)
    
    x = encoder(x)
    x = lstm(x)
    x = decoder(x)

    model = keras.Model(inputs=random_input, outputs=x, name=model_name)
    if weight is not None:
        model.load_weights(weight, by_name=True, skip_mismatch=True)
    return model

In [103]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    lr_schedule = keras.optimizers.schedules.CosineDecay(0.001, 0.9)
    model = build_model()
    model.compile(optimizer=keras.optimizers.Adam(lr_schedule),
                  loss=tf.keras.losses.MeanSquaredError())

model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Model: "encoder-decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_34 (InputLayer)       [(None, 1, 256, 483)]     0         
                                                                 
 en1 (Conv2D)                (None, 16, 128, 483)      176       
                                                                 
 en2 (Conv2D)                (None, 32, 64, 483)       5152      
                                                                 
 en3 (Conv2D)                (None, 64, 32, 483)       20544     